# TruGraph Tutorial: Instrumenting LangGraph Applications with OTel

This notebook demonstrates how to use TruGraph to instrument LangGraph applications for evaluation and monitoring.

## Overview

TruGraph provides:
- **Automatic detection** of LangGraph applications
- **Combined instrumentation** of both LangChain and LangGraph components
- **Multi-agent evaluation** capabilities
- **Automatic @task instrumentation** with intelligent attribute extraction

## Installation

First, make sure you have the required packages installed:


In [ ]:
# Install required packages
#!pip install trulens-apps-langgraph langgraph langchain-core langchain-openai langchain-community


## OpenTelemetry (OTel) Compatibility

TruGraph supports both traditional TruLens instrumentation and OpenTelemetry (OTel) tracing mode:

- **Traditional Mode** (default): Uses TruLens native instrumentation with `CombinedInstrument`
- **OTel Mode**: Uses OpenTelemetry spans for interoperability with existing telemetry stacks

To enable OTel mode, set the environment variable:
```python
import os
os.environ["TRULENS_OTEL_TRACING"] = "1"
```

⚠️ **Note**: When OTel tracing is enabled, TruGraph automatically detects the main method (`invoke` or `run`) and uses OTel-compatible instrumentation. The traditional instrumentation system is disabled in OTel mode.

## Basic Setup

Let's start by checking if LangGraph is available and importing the necessary components:


In [ ]:
import os


from trulens.core.session import TruSession

os.environ["TRULENS_OTEL_TRACING"] = "1"

session = TruSession()
session.reset_database()
# Check if LangGraph is available
try:
    from langgraph.graph import StateGraph, MessagesState, END
    from langchain_core.messages import HumanMessage, AIMessage
    from trulens.apps.langgraph import TruGraph
    print("✅ LangGraph and TruGraph are available!")
    LANGGRAPH_AVAILABLE = True
except ImportError as e:
    print(f"❌ LangGraph not available: {e}")
    print("Please install with: pip install langgraph")
    LANGGRAPH_AVAILABLE = False



## Example 1: Simple Multi-Agent Workflow

Let's create a basic multi-agent workflow with a researcher and writer:


In [ ]:
if LANGGRAPH_AVAILABLE:
    # Define agent functions
 
    def research_agent(state):
        """Agent that performs research on a topic."""
        messages = state.get("messages", [])
        if messages:
            last_message = messages[-1]
            if hasattr(last_message, "content"):
                query = last_message.content
            else:
                query = str(last_message)
            
            # Simulate research (in a real app, this would call external APIs)
            research_results = f"Research findings for '{query}': This is a comprehensive analysis of the topic."
            return {"messages": [AIMessage(content=research_results)]}
        
        return {"messages": [AIMessage(content="No research query provided")]}
    
    def writer_agent(state):
        """Agent that writes articles based on research."""
        messages = state.get("messages", [])
        if messages:
            last_message = messages[-1]
            if hasattr(last_message, "content"):
                research_content = last_message.content
            else:
                research_content = str(last_message)
            
            # Simulate article writing
            article = f"Article: Based on the research - {research_content[:100]}..."
            return {"messages": [AIMessage(content=article)]}
        
        return {"messages": [AIMessage(content="No research content provided")]}
    
    # Create the workflow
    workflow = StateGraph(MessagesState)
    workflow.add_node("researcher", research_agent)
    workflow.add_node("writer", writer_agent)
    workflow.add_edge("researcher", "writer")
    workflow.add_edge("writer", END)
    workflow.set_entry_point("researcher")
    
    # Compile the graph
    graph = workflow.compile()
    
    print("✅ Multi-agent workflow created successfully!")
    print(f"Graph type: {type(graph)}")
    print(f"Graph module: {graph.__module__}")
else:
    print("❌ Skipping workflow creation - LangGraph not available")


In [ ]:
tru_app = TruGraph(graph, app_name="simple graph api app")
with tru_app as recording:
    result = graph.invoke({"messages": [HumanMessage(content="Hello!")]})
    print(result)
session.force_flush()   

## Example 2: Test Auto-Detection

Let's test whether TruSession can automatically detect our LangGraph application:


## Automatic @task Detection

One of the key features of TruGraph is its ability to automatically detect and instrument functions decorated with LangGraph's `@task` decorator. This means you can use standard LangGraph patterns without any additional instrumentation code.

### How it works:

1. **Automatic Detection**: TruGraph automatically scans for functions decorated with `@task`
2. **Smart Attribute Extraction**: It intelligently extracts information from function arguments:
   - Handles `BaseChatModel` and `BaseModel` objects
   - Extracts data from dataclasses and Pydantic models
   - Skips non-serializable objects like LLM pools
   - Captures return values and exceptions
3. **Seamless Integration**: No additional decorators or code changes required

### Example Usage:

```python
from langgraph.func import task

@task  # This is automatically detected and instrumented by TruGraph
def my_agent_function(state, config):
    # Your agent logic here
    return updated_state
```

The instrumentation happens automatically when you create a TruGraph instance - no manual setup required!


### @task Example

Create a real LangGraph application using the `@task` decorator to see automatic instrumentation in action:


In [ ]:
if LANGGRAPH_AVAILABLE:
    """
    LangGraph Functional API Example with TruGraph

    This example demonstrates the correct usage of LangGraph's functional API
    with @task and @entrypoint decorators, showing how TruGraph automatically
    instruments these functions for evaluation and monitoring.

    Key concepts:
    - @task: Decorates functions that perform discrete units of work
    - @entrypoint: Decorates the main workflow function that orchestrates tasks
    - Tasks return futures that must be resolved with .result() or await
    - The entrypoint coordinates the overall workflow execution
    """

    import os
    from typing import Dict, Any, cast

    # Import LangGraph functional API components
    try:
        from langgraph.func import task, entrypoint
        from langgraph.checkpoint.memory import MemorySaver
        from langchain_core.runnables import RunnableConfig
        FUNCTIONAL_API_AVAILABLE = True
    except ImportError:
        print("⚠️ LangGraph Functional API not available")
        print("   Install with: pip install langgraph>=0.2.0")
        FUNCTIONAL_API_AVAILABLE = False
        RunnableConfig = Dict[str, Any]  # Fallback type

    # Import TruGraph for automatic instrumentation
    try:
        from trulens.apps.langgraph import TruGraph
        from trulens.core import TruSession
        TRUGRAPH_AVAILABLE = True
    except ImportError:
        print("⚠️ TruGraph not available")
        print("   Install with: pip install trulens-apps-langgraph")
        TRUGRAPH_AVAILABLE = False


    # Define task functions using @task decorator
    @task
    def analyze_query(query: str) -> Dict[str, Any]:
        """Analyze the user's query to understand intent and complexity."""
        print(f"  🔍 Analyzing query: '{query}'")
        
        # Simple analysis simulation - in practice this might call an LLM
        words = query.split()
        analysis = {
            "intent": "information_request" if "?" in query else "statement",
            "complexity": "high" if len(words) > 10 else "medium" if len(words) > 5 else "simple",
            "keywords": words[:3],  # First 3 words as keywords
            "sentiment": "neutral",
            "word_count": len(words)
        }
        
        print(f"  ✅ Analysis complete: {analysis}")
        return analysis


    @task
    def generate_response(query: str, analysis: Dict[str, Any]) -> str:
        """Generate a response based on the query and analysis."""
        print(f"  🤖 Generating response for: '{query[:50]}...'")
        
        keywords = analysis.get("keywords", [])
        complexity = analysis.get("complexity", "medium")
        
        if keywords:
            if complexity == "high":
                response = f"This is a complex question about {', '.join(keywords)}. Let me provide a comprehensive analysis: {query} involves multiple interconnected concepts that require careful consideration..."
            elif complexity == "medium":
                response = f"Regarding {', '.join(keywords)}, here's what you should know: This topic involves several key aspects that are important to understand..."
            else:
                response = f"About {', '.join(keywords)}: This is a straightforward topic that can be explained simply..."
        else:
            response = "I'd be happy to help you with information and analysis on any topic you're interested in."
        
        print(f"  ✅ Response generated: {response[:50]}...")
        return response


    @task
    def format_final_output(query: str, analysis: Dict[str, Any], response: str) -> Dict[str, Any]:
        """Format the final output with metadata."""
        print("  📝 Formatting final output...")
        
        formatted_output = {
            "original_query": query,
            "analysis_results": analysis,
            "generated_response": response,
            "metadata": {
                "processing_steps": ["analysis", "generation", "formatting"],
                "response_length": len(response),
                "keywords_found": len(analysis.get("keywords", [])),
                "complexity_level": analysis.get("complexity", "unknown")
            },
            "status": "completed"
        }
        
        print("  ✅ Formatting complete")
        return formatted_output


    # Define the main workflow using @entrypoint
    @entrypoint(checkpointer=MemorySaver())
    def intelligent_qa_workflow(input_data: Dict[str, Any]) -> Dict[str, Any]:
        """
        Main workflow that orchestrates query analysis and response generation.
        
        This is the entrypoint that coordinates all the @task functions.
        Each task returns a future that must be resolved with .result().
        """
        query = input_data.get("query", "")
        
        print(f"🎬 Starting intelligent Q&A workflow for: '{query[:50]}...'")
        
        # Step 1: Analyze the query (returns a future)
        analysis_future = analyze_query(query)
        
        # Step 2: Resolve the analysis future to get the actual result
        analysis = analysis_future.result()
        
        # Step 3: Generate response based on query and analysis (returns a future)
        response_future = generate_response(query, analysis)
        
        # Step 4: Resolve the response future
        response = response_future.result()
        
        # Step 5: Format the final output (returns a future)
        formatted_future = format_final_output(query, analysis, response)
        
        # Step 6: Resolve the final formatting future
        final_result = formatted_future.result()
        
        print(f"✅ Workflow completed successfully")
        return final_result

In [ ]:
test_input = {
        "query": "What is machine learning and how does it work in practice?"
    }
    
config = cast(RunnableConfig, {
    "configurable": {
        "thread_id": "test-thread-1"
    }
})

tru_task_app = TruGraph(intelligent_qa_workflow, app_name="Functional api app")
with tru_task_app as recording:
    direct_result = intelligent_qa_workflow.invoke(test_input, config)

session.force_flush()
print(f"\n📊 Direct Execution Results:")
print(f"  Query: {direct_result['original_query']}")
print(f"  Analysis: {direct_result['analysis_results']}")
print(f"  Response: {direct_result['generated_response'][:100]}...")
print(f"  Metadata: {direct_result['metadata']}")

### Testing OTel Compatibility

Now let's test TruGraph with OpenTelemetry tracing enabled:


In [ ]:
# Test OTel Mode Compatibility
if LANGGRAPH_AVAILABLE:
    import os
    
    print("🧪 Testing OTel Mode Compatibility:")
    
    # Note: Enable OTel tracing (commented out to avoid conflicts in tutorial)
    # Uncomment the next line to test OTel mode:
    os.environ["TRULENS_OTEL_TRACING"] = "1"
    
    # For demonstration, show how OTel would be enabled
    print("  To enable OTel mode:")
    print('  os.environ["TRULENS_OTEL_TRACING"] = "1"')
    print()
    
    # Create a fresh session for OTel testing
    from trulens.core.session import TruSession
    otel_session = TruSession()
    
    try:
        otel_tru_app = otel_session.App(graph, app_name="OTelCompatibilityTest")
        print(f"✅ TruGraph created successfully: {type(otel_tru_app)}")
        print(f"   OTel Mode would automatically:")
        print(f"   - Detect main method: invoke or run")
        print(f"   - Use OpenTelemetry spans instead of traditional instrumentation")
    except Exception as e:
        print(f"❌ Error creating TruGraph: {e}")
        
else:
    print("❌ Skipping OTel test - LangGraph not available")


In [ ]:
if LANGGRAPH_AVAILABLE:
    # Test the detection
    session = TruSession()
    session.reset_database()
    
    print("🔍 Testing LangGraph Detection:")
    print(f"  Module check: {graph.__module__.startswith('langgraph')}")
    print(f"  Type check: {session._is_langgraph_app(graph)}")
    print(f"  Has graph attr: {hasattr(graph, 'graph')}")
    print(f"  Has invoke method: {hasattr(graph, 'invoke')}")
    
    # Test automatic detection
    print("\n🎯 Testing Automatic Detection:")
    tru_app = session.App(graph, app_name="AutoDetectionTest")
    
    print(f"  Created type: {type(tru_app)}")
    print(f"  Is TruGraph: {isinstance(tru_app, TruGraph)}")
    print(f"  App name: {tru_app.app_name}")
    
    if isinstance(tru_app, TruGraph):
        print("✅ SUCCESS: Auto-detection worked!")
        
        # Test basic functionality
        test_input = {"messages": [HumanMessage(content="What is AI?")]}
        result = tru_app.app.invoke(test_input)
        print(f"  Test result: {result['messages'][-1].content[:50]}...")
        
        with tru_app as recording:
            result = tru_app.app.invoke(test_input)
        session.force_flush()
    else:
        print("❌ FAILED: Auto-detection didn't work")
        
    print("\n🎉 TruGraph Tutorial Complete!")
    print("For more examples, check the TruLens documentation at https://trulens.org/")
else:
    print("❌ Skipping detection test - LangGraph not available")
